 <font size="6"><b>Анализ SQL базы данных книжного магазина</b></font><br />

**Цель** - проанализировать содержимое БД на языке SQL согласно запросам заказчика.

**Описание данных**

Предоставлены 5 таблиц:


Таблица `books`

Содержит данные о книгах:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.


Таблица `authors`

Содержит данные об авторах:

 * author_id — идентификатор автора;
 * author — имя автора.
 

Таблица `publishers`

Содержит данные об издательствах:

 * publisher_id — идентификатор издательства;
 * publisher — название издательства;


Таблица `ratings`

Содержит данные о пользовательских оценках книг:

 * rating_id — идентификатор оценки;
 * book_id — идентификатор книги;
 * username — имя пользователя, оставившего оценку;
 * rating — оценка книги.


Таблица `reviews`

Содержит данные о пользовательских обзорах на книги:

 * review_id — идентификатор обзора;
 * book_id — идентификатор книги;
 * username — имя пользователя, написавшего обзор;
 * text — текст обзора.
 
 
 **Задачи**
 
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

#### Загрузка и изучение данных

In [33]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [34]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [35]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame: 
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [36]:
# формируем запрос и выводим данные
query_books = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [37]:
get_sql_data(query_books).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           5 non-null      int64 
 1   author_id         5 non-null      int64 
 2   title             5 non-null      object
 3   num_pages         5 non-null      int64 
 4   publication_date  5 non-null      object
 5   publisher_id      5 non-null      int64 
dtypes: int64(4), object(2)
memory usage: 368.0+ bytes


In [38]:
# формируем запрос и выводим данные
query_authors = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [39]:
get_sql_data(query_authors).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  5 non-null      int64 
 1   author     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [40]:
# формируем запрос и выводим данные
query_publishers = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [41]:
get_sql_data(query_publishers).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  5 non-null      int64 
 1   publisher     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


In [42]:
# формируем запрос и выводим данные
query_ratings = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query_ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [43]:
get_sql_data(query_ratings).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   rating     5 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 288.0+ bytes


In [44]:
# формируем запрос и выводим данные
query_reviews = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query_reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [45]:
get_sql_data(query_reviews).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  5 non-null      int64 
 1   book_id    5 non-null      int64 
 2   username   5 non-null      object
 3   text       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 288.0+ bytes


#### Посчитайте, сколько книг вышло после 1 января 2000 года

In [46]:
query1 = '''
            SELECT COUNT(book_id) AS book_cnt
            FROM books
            WHERE publication_date::date > '2000-01-01'
        '''

In [47]:
# формируем запрос и выводим данные
books_cnt = get_sql_data(query1)
books_cnt

,book_cnt
0,819


In [48]:
print('Количество книг, вышедших после 1 января 2000 года составило:', books_cnt.loc[0, 'book_cnt'])

Количество книг, вышедших после 1 января 2000 года составило: 819


#### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [64]:
query2 = '''
            WITH book_rating AS
            (SELECT book_id,
            AVG(rating) AS avg_rating
            FROM ratings
            GROUP BY book_id),

            book_review AS
            (SELECT book_id,
            COUNT(*) AS review_count
            FROM reviews
            GROUP BY book_id)
            
            
            SELECT b.title, b.book_id, avg_rating, review_count
            FROM books AS b
            LEFT JOIN book_rating AS b_rat ON b.book_id = b_rat.book_id
            LEFT JOIN book_review AS b_rev ON b.book_id = b_rev.book_id            
        '''

In [66]:
# формируем запрос и выводим данные
books_rv_rt = get_sql_data(query2)
books_rv_rt

,title,book_id,avg_rating,review_count
0,The Body in the Library (Miss Marple #3),652,4.500000,2.0
1,Galápagos,273,4.500000,2.0
2,A Tree Grows in Brooklyn,51,4.250000,5.0
3,Undaunted Courage: The Pioneering First Missio...,951,4.000000,2.0
4,The Prophet,839,4.285714,4.0
...,...,...,...,...
995,Alice in Wonderland,64,4.230769,4.0
996,A Woman of Substance (Emma Harte Saga #1),55,5.000000,2.0
997,Christine,148,3.428571,3.0
998,The Magicians' Guild (Black Magician Trilogy #1),790,3.500000,2.0


#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц

In [53]:
query3 = '''
            SELECT p.publisher,
                   COUNT(b.book_id) AS books_cnt                   
            FROM books AS b
            JOIN publishers AS p ON b.publisher_id=p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY books_cnt DESC
            LIMIT 1
        '''

In [54]:
# формируем запрос и выводим данные
publ = get_sql_data(query3)
publ

,publisher,books_cnt
0,Penguin Books,42


In [55]:
print('Издательство, выпустившее наибольшее наибольшее число книг толще 50 страниц:', publ.loc[0, 'publisher'])

Издательство, выпустившее наибольшее наибольшее число книг толще 50 страниц: Penguin Books


#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [56]:
query4 = '''
            SELECT a.author AS author,
                   ROUND(AVG(r.rating),1) AS avg_rating                   
            FROM ratings AS r
            JOIN books AS b ON b.book_id=r.book_id
            JOIN authors AS a ON b.author_id=a.author_id
            WHERE r.book_id IN ( SELECT book_id
                                 FROM ratings
                                 GROUP BY book_id
                                 HAVING COUNT(book_id) >= 50 )
            GROUP BY a.author_id
            ORDER BY avg_rating DESC
            LIMIT 1
        '''

In [57]:
# формируем запрос и выводим данные
author = get_sql_data(query4)
author

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.3


In [58]:
print('Автор с самой высокой средней оценкой книг:',author.loc[0, 'author'],' \nCредняя оценка:',author.loc[0, 'avg_rating'])

Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré  
Cредняя оценка: 4.3


#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [59]:
query5 = '''
            SELECT ROUND(AVG(rev_cnt),1) as avg_rev_cnt
            FROM ( SELECT username,
                   COUNT(review_id) as rev_cnt
                   FROM reviews
                   GROUP BY username
                   HAVING username IN ( SELECT username
                                        FROM ratings
                                        GROUP BY username
                                        HAVING COUNT(rating_id) > 48
                                        )
                            
                            
                  ) as reviews
        '''

In [60]:
# формируем запрос и выводим данные
avg_rev = get_sql_data(query5)
avg_rev

,avg_rev_cnt
0,24.0


In [61]:
print('Среднее количество обзоров от пользователей, которые поставили больше 48 оценок:', avg_rev.loc[0, 'avg_rev_cnt'])

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.0


#### Выводы

1. Количество книг, вышедших после 1 января 2000 года составило: *819*
2. Выведена *таблица* с количеством обзоров и средней оценкой для каждой из 1000 книг, хранящихся в БД.
3. Издательство, выпустившее наибольшее наибольшее число книг толще 50 страниц: *Penguin Books*
4. Автор с самой высокой средней оценкой книг (учитывались только книги с 50 и более оценками): *J.K. Rowling/Mary GrandPré. Cредняя оценка: 4.3*.
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок: *24.0*